# Using NLTK to get similar patents

In [38]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup
import urllib
import json
import numpy as np

import hashlib
import sqlite3
import urllib2
import time
import pickle as cPickle
import re
from collections import Counter

from os import listdir
from os.path import isfile, join

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

import pandas as pd
import sklearn
import numpy as np
import nltk
import re
import time
import codecs

import os.path

import pandas as pd
pd.options.display.max_columns = 200
pd.options.mode.chained_assignment = None

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from string import punctuation

from collections import Counter
import re
import numpy as np

from tqdm import tqdm_notebook
tqdm_notebook().pandas()

from sklearn.neighbors import NearestNeighbors

from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
def clean_text(text):
    stop_words = ['\x0c', '\n']
    for i in stop_words:
        text.replace(i, ' ')
    clean_text = re.sub('[^a-zA-Z]+', ' ', text)
    return clean_text.lower()
def tokenize_and_stem(text):
    tokens = nltk.word_tokenize(text)
    stemmer = nltk.stem.porter.PorterStemmer()
    return [i for i in [stemmer.stem(t) for t in tokens] if len(i) > 2]

In [11]:
onlyfiles = [f for f in listdir("./.cache/") if isfile(join("./.cache/", f))]
print onlyfiles

In [17]:
text = pd.DataFrame(columns = ["id", "content"])

for fname in onlyfiles:
    pURL = "./.cache/"+fname
    file_object  = open(pURL, "r") 
    r =  file_object.read()

    soup = BeautifulSoup(r,"lxml")
    
    try:
        abstr = fname
        content = soup.getText()
        
        text = text.append(pd.DataFrame([[abstr, content]],
                             columns=["id", "content"]),
                      ignore_index=True)            
    except:
        pass
        
    print fname


In [67]:
text.to_csv('./pandas.csv', index=False, encoding='utf-8')

In [68]:
text.shape

(287, 3)

In [26]:
text.head()

,id,content,content_clean
0,US3676584A,\n\nUS3676584A - Echo coincidence ultrasonic s...,us a echo coincidence ultrasonic scanning goo...
1,US3090030A,\n\nUS3090030A - Variable focus transducer \n ...,us a variable focus transducer google patents...
2,US3881164A,\n\nUS3881164A - Cross array ultrasonic transd...,us a cross array ultrasonic transducer google...
3,US5538004A,\n\nUS5538004A - Method and apparatus for tiss...,us a method and apparatus for tissue centered...
4,US5325860A,\n\nUS5325860A - Ultrasonic and interventional...,us a ultrasonic and interventional catheter a...


In [25]:
text['content_clean'] = text['content'].apply(clean_text) 

In [29]:
# Create vectorizer for content, max_df is set to 0.5, we only want 
# to include terms that appear in less than 50% of the documents (i.e. rare terms)
content_tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=0, max_features=200000,
               stop_words='english', use_idf=True, tokenizer=tokenize_and_stem)

In [33]:
tfidf_weights = content_tfidf_vectorizer.fit_transform(text['content_clean'])

In [34]:
tfidf_features = content_tfidf_vectorizer.get_feature_names()

In [35]:
# Function for returning the top_k features of a content

def get_top_features(rownum, weights, features, top_k=10):
    weight_vec = weights.toarray()[rownum,:]
    top_idx = np.argsort(weight_vec)[::-1][:top_k]
    return [features[i] for i in top_idx]



In [66]:
get_top_features(20, tfidf_weights, tfidf_features)

[u'counter',
 u'preset',
 u'focuss',
 u'termin',
 u'regist',
 u'shift',
 u'electro',
 u'deflect',
 u'convers',
 u'clock']

In [47]:
# Create the k-NN model using k=5
nn = NearestNeighbors(n_neighbors=10, algorithm='auto')
# Fit the models to the TF-IDF weights matrix
nn_fitted = nn.fit(tfidf_weights)

In [48]:
def find_nearest_patents(row, kNNmodel, tfidf_weights, tfidf_features, papers):
    keywords = get_top_features(row, tfidf_weights, tfidf_features)
    dist,idx = kNNmodel.kneighbors(tfidf_weights[row,:])
    idx = list(idx[0])
    return {'papers':papers.ix[idx], 'keywords':keywords}

In [65]:
find_nearest_patents(20, nn_fitted, tfidf_weights, tfidf_features, text)['papers']

,id,content,content_clean
20,US4019169A,\n\nUS4019169A - Ultrasonic wave transmitting ...,us a ultrasonic wave transmitting and receivi...
23,US4136325A,\n\nUS4136325A - Ultrasonic wave transmitting ...,us a ultrasonic wave transmitting and receivi...
61,US4080838A,\n\nUS4080838A - Method and apparatus for cont...,us a method and apparatus for controlling ult...
127,US4099419A,\n\nUS4099419A - Ultrasonic tomography apparat...,us a ultrasonic tomography apparatus google p...
27,US4368643A,\n\nUS4368643A - Ultrasonic imaging by radial ...,us a ultrasonic imaging by radial scan beams ...
266,US4075598,\n\nUS4075598A - Ultrasonic wave transmitting ...,us a ultrasonic wave transmitting and receivi...
255,US4075598A,\n\nUS4075598A - Ultrasonic wave transmitting ...,us a ultrasonic wave transmitting and receivi...
137,US3324452A,\n\nUS3324452A - Digital phase control system ...,us a digital phase control system for use in ...
5,US4161122A,\n\nUS4161122A - Apparatus for examining bodie...,us a apparatus for examining bodies through s...
17,US3919683,\n\nUS3919683A - Ultrasonic wave transmitting ...,us a ultrasonic wave transmitting and receivi...


In [57]:
a = find_nearest_patents(20, nn_fitted, tfidf_weights, tfidf_features, text)['papers']["id"]

In [64]:
closest = []
for k in a:
    closest.append(k)
closest

['US4019169A',
 'US4136325A',
 'US4080838A',
 'US4099419A',
 'US4368643A',
 'US4075598',
 'US4075598A',
 'US3324452A',
 'US4161122A',
 'US3919683']

In [62]:
find_nearest_patents(1, nn_fitted, tfidf_weights, tfidf_features, text)['keywords']

[u'electrod',
 u'navico',
 u'triod',
 u'termin',
 u'relay',
 u'tirn',
 u'network',
 u'conductor',
 u'frorn',
 u'rnean']